In [1]:
import pandas as pd
import numpy as np
import re

df = pd.read_parquet('output/sentiments.parquet')

# Dataset cleaning
df.dropna(inplace=True)
df = df[(df['review'].apply(lambda x: re.match(r'^[()\[\]{}\'!,.: ]+$', x) is None)) & (df['review'].apply(lambda x: len(x) > 1))]

df['sentiment'] = df['sentiment'].map({
    'Very Negative': -2,
    'Negative': -1,
    'Neutral': 0,
    'Positive': 1,
    'Very Positive': 2
})

#df.rename(columns={'game': 'SteamID'}, inplace=True)
#df = games[['game', 'SteamID']].merge(df, on='SteamID', how='inner').drop(columns=['SteamID'])
df['game'] = df['game'].astype('category')

df

,topic,review,embedding,sentiment,game
0,gameplay mechanics,Excellent game mechanics hampered by really sh...,"[0.02064609, 0.058051355, -0.15955217, -0.0282...",-1,1653100
1,gameplay mechanics,I love the narrator just talking in the backgr...,"[-0.01935661, 0.021597555, -0.18828672, -0.039...",1,1653100
2,gameplay mechanics,The game has received frequent and substantial...,"[0.030498626, 0.035088323, -0.17761268, -0.017...",1,1653100
3,gameplay mechanics,Heliotropism is a roguelike Tree simulator wit...,"[-0.005617, 0.06219709, -0.17552276, -0.042904...",1,1653100
4,narrator and audio,"the narrator is cool at first, then hyper anno...","[0.01688186, 0.010326986, -0.16718766, -0.0392...",-1,1653100
...,...,...,...,...,...
96560,music and sound,Хорошее музыкальное и звуковое сопровождение.,"[-0.035658363, -0.0012977426, -0.1295827, 0.02...",1,2300300
96561,music and sound,"I like the music, it's nice relaxing.","[-0.020432916, 0.03288259, -0.18042335, -0.001...",1,2300300
96562,music and sound,The music is nice relaxing.,"[-0.0041316957, 0.0568026, -0.18956739, 0.0036...",1,2300300
96563,price and value,The asking price might be a bit steep for what...,"[0.012826282, 0.04906256, -0.20521845, 0.02795...",0,2300300


In [2]:
from src.embed import get_embedding, NomicPrefix
from sklearn.metrics.pairwise import cosine_similarity

def get_similars(queries, embeddings, threshold):    
    query_embedding = get_embedding(queries)
    similarities = cosine_similarity(query_embedding, embeddings)
    
    indices = pd.DataFrame(data=np.argwhere(similarities > threshold), columns=['query', 'review'])
    indices['query'] = indices['query'].map(lambda x: queries[x])

    return indices.groupby('query')['review'].apply(list).to_dict()

In [3]:
citations_df = df.drop_duplicates(subset='review')
citation_embeddings = np.vstack(citations_df['embedding'].values)

#all_topics = df['topic'].unique()
#topics_embeddings = get_embedding(list(all_topics))
#pd.DataFrame(data=topics_embeddings, index=all_topics).to_parquet('output/topics.parquet')

all_topics = pd.read_parquet('output/topics.parquet')
topics_embeddings = all_topics.values
all_topics = all_topics.index

In [4]:
len(all_topics)

6449

In [5]:
#from yellowbrick.cluster import KElbowVisualizer
#from sklearn.cluster import KMeans
#import seaborn as sns
#import matplotlib.pyplot as plt
#from sklearn.manifold import TSNE

#plt.set_loglevel('error')

#model = KMeans()
#visualizer = KElbowVisualizer(model, k=range(5, 60, 2))

#topics_clustering_embeddings = get_embedding(list(all_topics), prefix=NomicPrefix.CLUSTERING)

#visualizer.fit(topics_clustering_embeddings)
#visualizer.show()

In [6]:
#kmeans = KMeans(n_clusters=visualizer.elbow_value_).fit_predict(topics_clustering_embeddings)
#tsne = TSNE(n_components=2, verbose=1, max_iter=500).fit_transform(topics_clustering_embeddings)

In [7]:
# Select 10 random clusters
#df_cluster = pd.DataFrame(data=tsne, columns=['x', 'y'])
#df_cluster['cluster'] = kmeans

#random_clusters = np.random.choice(np.unique(kmeans), 10)
#df_cluster = df_cluster[df_cluster['cluster'].isin([2, 5, 7, 8, 9, 14, 15, 25, 28, 30])]

#sns.scatterplot(data=df_cluster, x='x', y='y', hue='cluster', palette='tab10', s=30, legend=False)
#plt.savefig('output/out/clusters.pdf', bbox_inches='tight', format='pdf')
#plt.show()

In [8]:
def search_similars(queries):
    similars_by_text = get_similars(queries, citation_embeddings, 0.8)
    similars_by_topic = get_similars(queries, topics_embeddings, 0.85)

    results = {}
    for query in queries:
        sim_a = citations_df.iloc[similars_by_text.get(query, [])]

        included_topics = all_topics[similars_by_topic.get(query, [])]
        sim_b = df[df['topic'].isin(included_topics)]

        # Merge and drop duplicates
        sim = pd.concat([sim_a, sim_b]).drop_duplicates(subset='review')
        results[query] = sim[['review', 'sentiment', 'game']]
    
    return results

In [9]:
import json

queries: dict = json.load(open('data/queries.json'))

In [10]:
results = {
    category: search_similars(q)
    for category, q in queries.items()
}

In [11]:
results_df = pd.concat([
    pd.concat([d.assign(category=category, query=q) for q, d in cat_q.items()])
    for category, cat_q in results.items()
])

results_df['query'] = results_df['query'].astype('category')
results_df['category'] = results_df['category'].astype('category')

results_df

,review,sentiment,game,category,query
1191,Nice puzzle game,1,1875060,Gameplay Mechanics,Puzzle
1315,I like puzzles.,1,1875060,Gameplay Mechanics,Puzzle
1329,Nice puzzle game.,1,1875060,Gameplay Mechanics,Puzzle
1391,fun puzzle game,1,1875060,Gameplay Mechanics,Puzzle
1688,Lovely little puzzle game,1,1875060,Gameplay Mechanics,Puzzle
...,...,...,...,...,...
94157,"O jogo é muito bonitinho, o conceito é legal, ...",1,222730,Social and Educational Aspects,Educational Value
94158,"Eu daria pro meu filho ou filha jogar, se eu t...",1,222730,Social and Educational Aspects,Educational Value
96107,I could easily see this being the kind of educ...,1,2289390,Social and Educational Aspects,Educational Value
96108,An adorable management game with an ecological...,1,2289390,Social and Educational Aspects,Educational Value


In [12]:
def get_table(results, agg_fn):
    df = pd.DataFrame()
    for _, queries in results.items():
        for query, citations in queries.items():
            #df[(category, query)] = agg_fn(citations)
            df[query] = agg_fn(citations)
    #df.columns = pd.MultiIndex.from_tuples(df.columns, names=['category', 'query'])
    return df

counts = get_table(results, lambda x: x['game'].value_counts()).sort_index()
means = get_table(results, lambda x: x.groupby('game', observed=False)['sentiment'].mean()).sort_index()
std = get_table(results, lambda x: x.groupby('game', observed=False)['sentiment'].std()).sort_index()

# Select games with more than 10 hits in almost 2 categories
selected_games = counts[counts[counts >= 10].count(axis=1) > 1].index

# Hide values between -0.5 and 0.5 and mean support less than 10
means = means[(means.abs() > 0.5) & (counts >= 10)].loc[selected_games].dropna(how='all', axis=1).dropna(how='all', axis=0)

means

,Puzzle,Survival,Sandbox,Simulation,Storytelling and Narrative,Learning curve and Difficulty,Game pacing,Enjoyment and Fun,Relaxation and Calmness,Immersive Experience,...,Realism,Exploration and Discovery,Atmosphere and Ambiance,Graphics and Aesthetics,Sound and Music,Game length,Game Performance and Optimization,Controls and Input,Multiplayer,Educational Value
game,,,,,,,,,,,,,,,,,,,,,
1028590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.636364,NaN,NaN,...,0.720000,NaN,NaN,1.000000,NaN,NaN,-1.545455,NaN,NaN,NaN
1062090,NaN,NaN,0.843750,1.305882,NaN,NaN,-0.844828,1.563977,1.343648,NaN,...,NaN,NaN,1.291667,0.897260,0.917910,NaN,NaN,NaN,NaN,NaN
1133120,NaN,NaN,NaN,0.750000,NaN,NaN,NaN,1.476190,1.250000,NaN,...,-0.900000,NaN,NaN,-0.941176,NaN,NaN,-1.100000,NaN,NaN,NaN
1372320,0.833333,NaN,0.608696,NaN,NaN,NaN,NaN,1.476190,1.481884,NaN,...,NaN,NaN,1.169811,1.261745,1.300000,NaN,NaN,NaN,NaN,NaN
1390190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.641304,1.523256,NaN,...,NaN,NaN,1.458333,1.250000,0.916667,NaN,NaN,NaN,NaN,0.777778
1539580,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.971429,1.013889,NaN,...,NaN,NaN,1.400000,NaN,NaN,NaN,-1.357143,NaN,NaN,NaN
1575510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.333333,NaN,...,NaN,NaN,NaN,1.363636,NaN,NaN,NaN,NaN,NaN,NaN
1593030,0.789474,NaN,NaN,NaN,NaN,NaN,NaN,1.431818,1.445769,NaN,...,NaN,NaN,1.420000,1.364706,1.216867,NaN,NaN,NaN,NaN,1.266667
1837330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.166667,1.235294,NaN,NaN,NaN,NaN,NaN


In [13]:
games = pd.read_csv('./data/Game List - Final.csv')
games = games.dropna(subset=['SteamID']).rename(columns={'List (merge)': 'game'})
games = games[games['Final decision'] == 'Yes']

games['SteamID'] = games['SteamID'].astype(int).astype(str)
games.set_index('SteamID', inplace=True)
games = games[['game', 'LongCode']]
games.rename(columns={'LongCode': 'code'}, inplace=True)
games = games.loc[selected_games]

games.to_latex('output/games.tex')

id_code_map = games['code'].to_dict()

games

,game,code
game,,
1028590,The Wilds,Wilds
1062090,Timberborn,Timberborn
1133120,Ecosystem,Ecosys
1372320,Cloud Gardens,CloudG
1390190,APICO,Apico
1539580,Bird,Bird
1575510,Seedlings,Seedlings
1593030,Terra Nil,TerraNil
1837330,Gibbon: Beyond the Trees,Gibbon


In [14]:
from itertools import combinations
from collections import defaultdict

results_df = results_df[results_df['game'].isin(selected_games)]
results_df['game'] = results_df['game'].map(id_code_map)
grouped = results_df.groupby('query', observed=True)
cooccurrence = defaultdict(int)

# Count co-occurrences of reviews between query pairs
for (query1, reviews1), (query2, reviews2) in combinations(grouped, 2):
    shared_reviews = pd.merge(reviews1, reviews2, on='review', how='inner')
    cooccurrence[(query2, query1)] = shared_reviews

cooccurrence = pd.DataFrame([
    {'query1': q1, 'query2': q2, 'shared_reviews': count}
    for (q1, q2), count in cooccurrence.items()
])

cooccurrence = cooccurrence[cooccurrence['shared_reviews'].apply(lambda x: len(x) >= 10)]

cooccurrence['n'] = cooccurrence['shared_reviews'].apply(len)
cooccurrence['mean'] = cooccurrence['shared_reviews'].apply(lambda x: np.mean(x['sentiment_x']))
cooccurrence['top-3'] = cooccurrence['shared_reviews'].apply(lambda x: x.groupby('game_x', observed=True)['sentiment_x'].mean().nlargest(3).index.tolist())

cooccurrence

/tmp/ipykernel_1166766/225337440.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df['game'] = results_df['game'].map(id_code_map)


,query1,query2,shared_reviews,n,mean,top-3
33,Graphics and Aesthetics,Atmosphere and Ambiance,...,245,1.310204,"[Bird, Equi, Endling]"
40,Relaxation and Calmness,Atmosphere and Ambiance,...,56,1.535714,"[Equi, CloudG, Reus2]"
58,Map customization,Character customization,...,52,1.076923,"[BeeSim, Timberborn, Wolf-AE]"
91,Enjoyment and Fun,Educational Value,...,14,1.785714,"[Wolf-AE, Wolf-1]"
102,Realism,Educational Value,...,15,1.600000,"[Wolf-AE, BeeSim]"
133,Graphics and Aesthetics,Enjoyment and Fun,...,17,1.705882,"[Wolf-1, Timberborn, Wolf-AE]"
137,Multiplayer,Enjoyment and Fun,...,18,1.444444,"[Wolf-1, Eco, Wolf-AE]"
139,Realism,Enjoyment and Fun,...,34,1.823529,"[Wolf-AE, Wolf-1, BeeSim]"
140,Relaxation and Calmness,Enjoyment and Fun,...,1051,1.437678,"[Wolf-AE, Equi, Apico]"
180,Game pacing,Game length,...,43,-0.093023,"[Endling, Paperbark, Reus]"


In [20]:
top_3 = pd.DataFrame(
    index=pd.MultiIndex.from_frame(cooccurrence[['query1', 'query2']]),
    columns=['1', '2', '3'],
    data=cooccurrence['top-3'].tolist()
).fillna('').sort_index()

top_3.style.to_latex(
    buf='output/out/top-3.tex',
    position='t',
    position_float='centering',
    caption='Top 3 games with highest sentiment for each query pair',
    label='tab:top-3',
    hrules=True,
)

top_3

1           2  \
query1                  query2                                          
Enjoyment and Fun       Educational Value         Wolf-AE      Wolf-1   
Game pacing             Game length               Endling   Paperbark   
Graphics and Aesthetics Atmosphere and Ambiance      Bird        Equi   
                        Enjoyment and Fun          Wolf-1  Timberborn   
Map customization       Character customization    BeeSim  Timberborn   
Multiplayer             Enjoyment and Fun          Wolf-1         Eco   
Realism                 Educational Value         Wolf-AE      BeeSim   
                        Enjoyment and Fun         Wolf-AE      Wolf-1   
                        Graphics and Aesthetics   DNectar      Meadow   
Relaxation and Calmness Atmosphere and Ambiance      Equi      CloudG   
                        Enjoyment and Fun         Wolf-AE        Equi   
                        Graphics and Aesthetics     Apico      BeeSim   
                        Puzzle                   TerraNil      CloudG   
Replayability           Game length                   Eco    Shelter2   
Simulation              Realism                   DNectar     Wolf-AE   
Sound and Music         Graphics and Aesthetics       Eco    Preserve   
                        Relaxation and Calmness    CloudG      Meadow   

                                                        3  
query1                  query2                             
Enjoyment and Fun       Educational Value                  
Game pacing             Game length                  Reus  
Graphics and Aesthetics Atmosphere and Ambiance   Endling  
                        Enjoyment and Fun         Wolf-AE  
Map customization       Character customization   Wolf-AE  
Multiplayer             Enjoyment and Fun         Wolf-AE  
Realism                 Educational Value                  
                        Enjoyment and Fun          BeeSim  
                        Graphics and Aesthetics   Wolf-AE  
Relaxation and Calmness Atmosphere and Ambiance     Reus2  
                        Enjoyment and Fun           Apico  
                        Graphics and Aesthetics    CloudG  
                        Puzzle                   Preserve  
Replayability           Game length                  Equi  
Simulation              Realism                    Wolf-1  
Sound and Music         Graphics and Aesthetics    CloudG  
                        Relaxation and Calmness      Paws

In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns

fig, axs = plt.subplots(1, 2, figsize=(16, 8))

sns.heatmap(
    cooccurrence.pivot(index='query1', columns='query2', values='n'),
    annot=True,
    cmap='Spectral_r',
    fmt='.0f',
    ax=axs[0],
    center=0
)

sns.heatmap(
    cooccurrence.pivot(index='query1', columns='query2', values='mean'),
    annot=True,
    fmt='.2f',
    robust=True,
    cmap='Spectral_r',
    ax=axs[1],
    center=0,
)

axs[0].set_title('Number of co-occurrenced citations')
axs[1].set_title('Mean sentiment of co-occurrenced citations')
axs[0].set_ylabel('')
axs[0].set_xlabel('')
axs[1].set_ylabel('')
axs[1].set_xlabel('')

os.makedirs('output/out', exist_ok=True)
plt.tight_layout()
plt.savefig('output/out/cooccurrences.pdf', format='pdf', bbox_inches='tight')
plt.show()


In [ ]:
means.index = means.index.map(id_code_map)

# Sort columns by non-null values
means = means[means.count().sort_values(ascending=True).index]

# Sort index by non-null values
means = means.loc[means.count(axis=1).sort_values(ascending=False).index]

plt.figure(figsize=(15, 10))
sns.heatmap(means.T, annot=True, cmap='Spectral_r', fmt='.1f')
plt.xlabel('Game')
plt.ylabel('Query')
plt.savefig('output/out/mean-sentiment.pdf', format='pdf', bbox_inches='tight')

plt.show()